In [1]:
# 1. Imports

import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob

from moviepy.editor import VideoFileClip, concatenate_videoclips

In [2]:
# 2. Pre-Processing : Camera Calibration

 # import our camera calibration images 
images = glob.glob('./camera_cal/calibration*.jpg')

 # intialize our chess board variables
nx = 9
ny = 6
objpoints = []
imgpoints = []
objp = np.zeros((ny*nx,3), np.float32) #create a matrix to represent our objpoints
objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2) # populate the matrix with values for columns/rows

 # detect corners in each image
for file in images:
    img = mpimg.imread(file) # read in our image
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) # convert our image to grayscale
    ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None) # detect chessboard corners in grayed image
    if ret == True: # if correct corners were found
        imgpoints.append(corners) # add corners to our image points
        objpoints.append(objp) # add object points corresponding to image points
        img = cv2.drawChessboardCorners(img, (nx,ny), corners, ret) # Draw image for fun. Could be delted, or left for trouble shooting. can take time to compute.
        
 # calculate our undistortion coefficients
imshape = mpimg.imread(images[0]).shape[0:2] # call shape on the first image
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, imshape, None, None) # perform camera calibration function
RET, MTX, DIST, RVECS, TVECS = ret, mtx, dist, rvecs, tvecs # store these in unlikely to be overwritten variables

 # create function for use in pipeline
def cal_undistort(img):
    undist = cv2.undistort(img, MTX, DIST, None, MTX)
    return undist        

In [3]:
# Line class to keep data between video frames

class Lane():
    def __init__(self):
        # Number of frames to aggregate
        self.n = 0
        # Was the line detected last iteration?
        self.detected = False
        # x values of the last n fits of the line
        self.recent_xfitted = []
        # average x values of the fitted line over the last n iterations
        self.bestx = None
        # polynomial coefficients averaged over the last n iterations
        self.best_fit = None
        # polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]
        # radius of curvature of the line in some units
        self.radius_of_curvature = None
        # distance in meters of vehicle center from the line
        self.line_base_pos = None
        # difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float')
        # x values for detected line pixels
        self.allx = None
        # y values for detectedline pixels
        self.ally = None
        # flag for whether window method is required
        self.flag = 0
        # Crazy Count. Number of times we have recieved a crazy value compared to prior values
        self.crazycount = 0 
        
        self.x = []
        self.y = []
        
        self.recent_curve = []
    
    def curve_average(self):
        return 1
        
        
    def fit_best(self):
        if self.n <=8 : 
            self.n+=1
            self.best_fit = np.array(np.average(self.recent_xfitted, axis = 0))
        else:
            recent = np.array(self.recent_xfitted[-self.n:-1])
            self.best_fit = np.average(recent, axis = 0)
        return 1
    def crazy_check(self):
        if self.n >= 5:
            if ((self.current_fit[0] >= self.best_fit[0]*1.5) &  (self.current_fit[1] >= self.best_fit[1]*1.5)):
                self.crazycount +=1
                self.current_fit = self.recent_xfitted[-1]
            elif (self.current_fit[0] <= self.best_fit[0]/1.5) & (self.current_fit[1] <= self.best_fit[1]/1.5):
                self.crazycount +=1
                self.current_fit = self.recent_xfitted[-1]
            #else:
                #self.crazycount -=1

            if self.crazycount >= 3:
                self.flag = 0
                self.crazycount = 0
                print("IRRATIC BEHAVIOR DETECTED!")

Right_Lane = Lane()
Left_Lane = Lane()

In [4]:
# Binary Masking Functions

def extract_rgb_channels(img):
    r_channel = img[:,:,0]
    g_channel = img[:,:,1]
    b_channel = img[:,:,2]
    return r_channel, g_channel, b_channel 

def extract_hls_channels(img):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    h_channel = hls[:,:,0]
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    return h_channel, l_channel, s_channel 

# not used
def sobel_abs(img, sobel_kernel=3, orient='x'):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    if orient =='x':
        sobel=cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize = sobel_kernel)
    else:
        sobel=cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize = sobel_kernel)
    abs_sobel = np.absolute(sobel)
    return abs_sobel

# not used
def sobel_abs_thresh(img, sobel_kernel=3, thresh=(0,255), orient='x'):
    abs_sobel = sobel_abs(img, sobel_kernel=sobel_kernel, orient=orient)
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return binary_output

# not used
def sobel_dir_thresh(img, sobel_kernel=3, thresh=(0, (np.pi/2))):
    abs_sobelx = sobel_abs(img, sobel_kernel=sobel_kernel, orient='x')
    abs_sobely = sobel_abs(img, sobel_kernel=sobel_kernel, orient='y')
    soTan = np.arctan2(abs_sobely, abs_sobelx)
    tanbi = np.zeros_like(soTan)
    tanbi[(soTan > thresh[0]) & (soTan <= thresh[1])] = 1
    return tanbi

# not used
def sobel_mag_thresh(img, sobel_kernel=3, thresh=(0,255)):
    abs_sobelx = sobel_abs(img, sobel_kernel=sobel_kernel, orient='x')
    abs_sobely = sobel_abs(img, sobel_kernel=sobel_kernel, orient='y')
    abs_sobelxy = np.absolute(np.sqrt(np.square(abs_sobelx) + np.square(abs_sobely)))    
    scaled_sobelxy = np.uint8(255*abs_sobelxy/np.max(abs_sobelxy))
    sxybinary = np.zeros_like(scaled_sobelxy)
    sxybinary[(scaled_sobelxy >= thresh[0]) & (scaled_sobelxy <= thresh[1])] = 1
    return sxybinary


# Used
def compute_binary(img):
    
    '''
    R = img[:,:,0] # extract red channel
    B = img[:,:,2] # extract blue channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    L = img[:,:,1] # extract light channel
    S = img[:,:,2] # extract saturation channel
    '''
    H, L, S = extract_hls_channels(img)
    R, G, B = extract_rgb_channels(img)

    img_mag_thresh = sobel_mag_thresh(img, sobel_kernel = 3, thresh=(50,255))
    output_binary = output_binary = (((B<=50)|L>=100)&(R>=200) | ((H < 20) & (L < 100))&((R>200)|(img_mag_thresh==1)))
    Rmod = np.zeros_like(R)
    Rmod[(R>=100)&(S>=100) | (L>200)&(B>150)]=1
    return Rmod

In [5]:
# Image Warping Functions

def warp(img):
    img_size = (img.shape[1], img.shape[0]) # in the from X, Y
    # use empirically derived transform values
    src = np.float32([
        [1072, 690],
        [247, 690],
        [708, 460],
        [577, 460]
    ])
    dst = np.float32([
        [900, 700],
        [400, 700],
        [900, 200],
        [400, 200]
    ])
    
    M = cv2.getPerspectiveTransform(src, dst)  # Matrix for warping image
    Minv = cv2.getPerspectiveTransform(dst, src) # Martix for un-warping image
    warped = cv2.warpPerspective(img, M, img_size) # Warp our image
    return warped, M, Minv
    

In [6]:
# Find Lane Lines Window


MARGIN = 100
MINPIX = 50
def find_lanes_window(img):
    
    imshape = img.shape
    histogram = np.sum(img[imshape[0]//2:,:], axis=0) # create histogram of image
    out_img = np.dstack((img, img, img))*255 # create placeholder for out image
    
    midpoint = np.int(histogram.shape[0]/2) # find our image midpoint
    leftx_base = np.argmax(histogram[:midpoint]) # find the most likely left lane line center
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint # find the most likely right lane line center

    nwindows = 9 # number of windows to divide picture into
    window_height = np.int(imshape[0]/nwindows) # height of each window
    
    nonzero = img.nonzero() # find all nonzero points in our image
    nonzeroy = np.array(nonzero[0]) # y nonzero indices in our image
    nonzerox = np.array(nonzero[1]) # x nonzero indices in our image
    
    leftx_current = leftx_base
    rightx_current = rightx_base
    
    margin = MARGIN # the horixontal radius of our window
    minpix = MINPIX # the minimum pixels that will cause window center to be recalculated
    
    left_lane_inds = [] # list for storing left and right indices
    right_lane_inds = []
    
    for window in range(nwindows):
        # calculate window boundaries
        win_y_low = imshape[0] - (window+1)*window_height
        win_y_high = imshape[0] - (window)*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # draw windows for visualizations - can delete for production
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 

        # identify nonzero pixels within our windows
        windowed_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        windowed_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        
        # append these indices to our list
        left_lane_inds.append(windowed_left_inds)
        right_lane_inds.append(windowed_right_inds)
        
        # check for new center lins
        if len(windowed_left_inds) > minpix : 
            leftx_current = np.int(np.mean(nonzerox[windowed_left_inds]))
        if len(windowed_right_inds) > minpix : 
            rightx_current = np.int(np.mean(nonzerox[windowed_right_inds]))
            
     # preprocess our indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)
    
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
     # fit a second order polynomial
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    Left_Lane.current_fit = left_fit
    Right_Lane.current_fit = right_fit
    
    print(Left_Lane.current_fit, 'Window left lane fit')

    
    return 1

In [7]:
# Find lanes in a second+ image

def find_lanes(img):
    
    # Call first lane if Lanes.flag == no-go,
    # Else:
    if Left_Lane.flag == 0: 
        find_lanes_window(img)
        Left_Lane.flag = 1
    else :    
         # Pull left_fit, right_fit from Lanes.class
         # assuming we have a binary warped image
        nonzero = img.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        margin = MARGIN
        
        left_fit = Left_Lane.current_fit
        right_fit = Right_Lane.current_fit
         
        print(left_fit, 'Normal Lane left fit')


         # calculate our new left/right lane centers based off of previous pictures lane curvature
        left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
        right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin))) 

         # extract left/right lane pixel positions
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds]
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds]
        #print (left)
         # calculate a new polynomial
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)
        
        
        Left_Lane.x = leftx
        Left_Lane.y = lefty
        Right_Lane.x = rightx
        Right_Lane.y = righty
        
        Left_Lane.current_fit = left_fit
        Right_Lane.current_fit = right_fit
        Left_Lane.flag = 1
         # save polynomial to Class
    
    return 1

In [8]:
# Find Curvature of lanes

def find_curvature():
    # pull info from most recent Lanes.class()
    # perform some calculation
    # return

    
    ploty = np.linspace(0, 719, num=720)
    y_eval = np.max(ploty)
    
    left_fit_cr = Left_Lane.current_fit
    right_fit_cr = Right_Lane.current_fit

    ym_per_pix = 30/720
    xm_per_pix = 3.7/700
    
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    Left_Lane.radius_of_curvature = left_curverad/2
    
    print (left_curverad, "m: Left Turn Clyde", right_curverad, "m: Right turn Clyde ")
    

In [9]:
# Draw Lane Lines on orriginal Image

def draw_lanes(undistorted_img, warped_binary, Minv):
    
     # create image to draw lines on
    warp_zero = np.zeros_like(warped_binary).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    

    
    Left_Lane.recent_xfitted.append(Left_Lane.current_fit)
    Right_Lane.recent_xfitted.append(Right_Lane.current_fit)
     
    print(Left_Lane.current_fit, 'Left_Lane.current_fit')
    
    Left_Lane.fit_best()
    Right_Lane.fit_best()
       
    Left_Lane.crazy_check()
    Right_Lane.crazy_check()
     
    left_fit = Left_Lane.best_fit
    right_fit = Right_Lane.best_fit
    print(Left_Lane.best_fit)
    print(left_fit, 'draw lines left_fit')
    #left_fit = Left_Lane.current_fit
    #right_fit = Right_Lane.current_fit
    

    #print(Left_Lane.recent_xfitted)

    ploty = np.linspace(0, 719, num=720) # generate a line to perform fits on
    
     # grab left_fitx from Lanes.class
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    #print(left_fitx[0], ' the current left lane 0 point')
    xm_per_pix = 3.7/700
    left_distance = np.absolute(undistorted_img.shape[1]//2 - left_fitx[-1])*xm_per_pix
    right_distance = np.absolute(right_fitx[-1] - undistorted_img.shape[1]//2 )*xm_per_pix
     
    print(' %.1fm from the left lane line' %left_distance)
    print(' %.1fm from the right lane line' %right_distance)
    
     # format x & y points for cv2.fillPolly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    
     # draw lane onto warped blank
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255,0))
    
     # warp blank back to orriginal space
    newwarp = cv2.warpPerspective(color_warp, Minv, (undistorted_img.shape[1], undistorted_img.shape[0]))
     # combine overlay and orriginal image
    result = cv2.addWeighted(undistorted_img, 1, newwarp, 0.3, 0)
    
    strang = ("Curvature: %.1fm" %Left_Lane.radius_of_curvature)
    dleft = ("Distance from left lane: %.1fm" %left_distance)
    dright = ("Distance from right lane: %.1fm" %right_distance)
    cv2.putText(result, strang, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, 500)
    cv2.putText(result, dleft, (50,100), cv2.FONT_HERSHEY_SIMPLEX, 1, 200)
    cv2.putText(result, dright, (50,10), cv2.FONT_HERSHEY_SIMPLEX, 1, 200)
    
    return result
    

In [10]:
#in_image = mpimg.imread('./test_images/straight_lines2.jpg')

def pipeline(inMage):
    #vid_img_in = mpimg.imread(inMage) # read in image from video
    undistorted_img = cal_undistort(inMage) # undistort image from camera perspective
    binary_img = compute_binary(undistorted_img) # perform binization of image
    warped_binary, M, Minv = warp(binary_img) # perform image translation/warp for different perspective
    find_lanes(warped_binary) # Find lane lines, store to Lanes.class
    find_curvature() # Find Curvature using Lanes.class
    product_image = draw_lanes(undistorted_img, warped_binary, Minv)
    return product_image

#product_image = pipeline(in_image)
#plt.imshow(product_image)
#plt.show()


In [12]:
from cv2 import VideoCapture



ret = True
cap = cv2.VideoCapture('project_video.mp4')
ret, frame = cap.read()


i = 1                        
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter('./output_images/video_out.avi', fourcc, 24.0, size, isColor=True)


while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        
        #Copy and write frame
        
        frame = pipeline(frame)
        
        f2 = frame.copy()
        out.write(f2)
        
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

out.release()
cap.release()
cv2.destroyAllWindows()
print('true')

[ -3.07973284e-04   4.09106303e-01   3.07875081e+02] Window left lane fit
2009.00332247 m: Left Turn Clyde 2289.51819137 m: Right turn Clyde 
[ -3.07973284e-04   4.09106303e-01   3.07875081e+02] Left_Lane.current_fit
[ -3.07973284e-04   4.09106303e-01   3.07875081e+02]
[ -3.07973284e-04   4.09106303e-01   3.07875081e+02] draw lines left_fit
 1.0m from the left lane line
 1.6m from the right lane line
[ -3.07973284e-04   4.09106303e-01   3.07875081e+02] Normal Lane left fit
1990.70109269 m: Left Turn Clyde 2323.64050641 m: Right turn Clyde 
[ -3.11224558e-04   4.10626979e-01   3.07968935e+02] Left_Lane.current_fit
[ -3.09598921e-04   4.09866641e-01   3.07922008e+02]
[ -3.09598921e-04   4.09866641e-01   3.07922008e+02] draw lines left_fit
 1.0m from the left lane line
 1.6m from the right lane line
[ -3.11224558e-04   4.10626979e-01   3.07968935e+02] Normal Lane left fit
2088.22664662 m: Left Turn Clyde 2268.63637391 m: Right turn Clyde 
[ -2.95701590e-04   4.06414441e-01   3.07655441e+0

[ -4.25074391e-04   5.47435123e-01   2.62962628e+02] Left_Lane.current_fit
[ -4.71269916e-04   5.60791372e-01   2.66290247e+02]
[ -4.71269916e-04   5.60791372e-01   2.66290247e+02] draw lines left_fit
 1.1m from the left lane line
 1.6m from the right lane line
[ -4.25074391e-04   5.47435123e-01   2.62962628e+02] Normal Lane left fit
1788.8426099 m: Left Turn Clyde 2177.42358083 m: Right turn Clyde 
[ -3.91830138e-04   5.26034760e-01   2.64586043e+02] Left_Lane.current_fit
[ -4.65288559e-04   5.61069868e-01   2.65310875e+02]
[ -4.65288559e-04   5.61069868e-01   2.65310875e+02] draw lines left_fit
 1.1m from the left lane line
 1.6m from the right lane line
[ -3.91830138e-04   5.26034760e-01   2.64586043e+02] Normal Lane left fit
1903.36059079 m: Left Turn Clyde 2135.69513769 m: Right turn Clyde 
[ -3.58485693e-04   5.01387605e-01   2.66895986e+02] Left_Lane.current_fit
[ -4.54340940e-04   5.57661930e-01   2.64695686e+02]
[ -4.54340940e-04   5.57661930e-01   2.64695686e+02] draw lines l

[ -2.89681139e-04   3.68666958e-01   2.94605703e+02] draw lines left_fit
 1.2m from the left lane line
 1.5m from the right lane line
[ -3.01540879e-04   3.65564340e-01   2.94636471e+02] Normal Lane left fit
1931.86912393 m: Left Turn Clyde 2258.49114381 m: Right turn Clyde 
[ -3.08605091e-04   3.71266673e-01   2.93587199e+02] Left_Lane.current_fit
[ -2.89629938e-04   3.66336427e-01   2.94527491e+02]
[ -2.89629938e-04   3.66336427e-01   2.94527491e+02] draw lines left_fit
 1.2m from the left lane line
 1.5m from the right lane line
[ -3.08605091e-04   3.71266673e-01   2.93587199e+02] Normal Lane left fit
1981.93527308 m: Left Turn Clyde 2335.77911682 m: Right turn Clyde 
[ -2.98880001e-04   3.63794847e-01   2.91263429e+02] Left_Lane.current_fit
[ -2.93580593e-04   3.66723407e-01   2.94307579e+02]
[ -2.93580593e-04   3.66723407e-01   2.94307579e+02] draw lines left_fit
 1.2m from the left lane line
 1.5m from the right lane line
[ -2.98880001e-04   3.63794847e-01   2.91263429e+02] Norma

1596.48554394 m: Left Turn Clyde 1958.74786055 m: Right turn Clyde 
[ -4.25886094e-04   5.02400744e-01   2.86376989e+02] Left_Lane.current_fit
[ -4.30859638e-04   4.95790121e-01   2.87418578e+02]
[ -4.30859638e-04   4.95790121e-01   2.87418578e+02] draw lines left_fit
 1.2m from the left lane line
 1.5m from the right lane line
[ -4.25886094e-04   5.02400744e-01   2.86376989e+02] Normal Lane left fit
1616.15011451 m: Left Turn Clyde 1966.06628924 m: Right turn Clyde 
[ -4.20566210e-04   5.01800679e-01   2.86099047e+02] Left_Lane.current_fit
[ -4.29470190e-04   4.96850665e-01   2.87199767e+02]
[ -4.29470190e-04   4.96850665e-01   2.87199767e+02] draw lines left_fit
 1.2m from the left lane line
 1.5m from the right lane line
[ -4.20566210e-04   5.01800679e-01   2.86099047e+02] Normal Lane left fit
1655.69414419 m: Left Turn Clyde 1979.26140761 m: Right turn Clyde 
[ -4.09788204e-04   4.99618781e-01   2.86268781e+02] Left_Lane.current_fit
[ -4.26607479e-04   4.97071531e-01   2.87004777e+

[ -3.58787333e-04   4.36403219e-01   2.94641092e+02]
[ -3.58787333e-04   4.36403219e-01   2.94641092e+02] draw lines left_fit
 1.1m from the left lane line
 1.6m from the right lane line
[ -3.27370329e-04   4.27870682e-01   2.86021309e+02] Normal Lane left fit
1776.93576988 m: Left Turn Clyde 2380.80731771 m: Right turn Clyde 
[ -3.64724418e-04   4.56371369e-01   2.81802166e+02] Left_Lane.current_fit
[ -3.56410037e-04   4.37455849e-01   2.92644522e+02]
[ -3.56410037e-04   4.37455849e-01   2.92644522e+02] draw lines left_fit
 1.1m from the left lane line
 1.6m from the right lane line
[ -3.64724418e-04   4.56371369e-01   2.81802166e+02] Normal Lane left fit
1670.74656414 m: Left Turn Clyde 2375.01217074 m: Right turn Clyde 
[ -3.95940208e-04   4.76675695e-01   2.80546156e+02] Left_Lane.current_fit
[ -3.57861368e-04   4.41486826e-01   2.90337628e+02]
[ -3.57861368e-04   4.41486826e-01   2.90337628e+02] draw lines left_fit
 1.1m from the left lane line
 1.6m from the right lane line
[ -3.

...

[  1.38792800e-04  -3.19609888e-01   5.73418499e+02] draw lines left_fit
 1.2m from the left lane line
 1.5m from the right lane line
[  1.46457046e-04  -3.19190303e-01   5.70533181e+02] Normal Lane left fit
4178.35204811 m: Left Turn Clyde 2531.54068422 m: Right turn Clyde 
[  1.37205463e-04  -3.17218589e-01   5.73614124e+02] Left_Lane.current_fit
[  1.37556024e-04  -3.15193203e-01   5.71057567e+02]
[  1.37556024e-04  -3.15193203e-01   5.71057567e+02] draw lines left_fit
 1.2m from the left lane line
 1.5m from the right lane line
[  1.37205463e-04  -3.17218589e-01   5.73614124e+02] Normal Lane left fit
5189.57097982 m: Left Turn Clyde 2492.50737688 m: Right turn Clyde 
[  1.08812168e-04  -2.97192982e-01   5.70720337e+02] Left_Lane.current_fit
[  1.37550414e-04  -3.12797162e-01   5.69619662e+02]
[  1.37550414e-04  -3.12797162e-01   5.69619662e+02] draw lines left_fit
 1.2m from the left lane line
 1.5m from the right lane line
[  1.08812168e-04  -2.97192982e-01   5.70720337e+02] Norma

[  1.57917919e-04  -2.45314949e-01   5.29169721e+02] draw lines left_fit
 1.1m from the left lane line
 1.5m from the right lane line
[  1.64125185e-04  -2.66643892e-01   5.33377005e+02] Normal Lane left fit
3350.73820043 m: Left Turn Clyde 2912.39793061 m: Right turn Clyde 
[  1.63500238e-04  -2.60433308e-01   5.27890445e+02] Left_Lane.current_fit
[  1.63005079e-04  -2.47802834e-01   5.27438958e+02]
[  1.63005079e-04  -2.47802834e-01   5.27438958e+02] draw lines left_fit
 1.1m from the left lane line
 1.5m from the right lane line
[  1.63500238e-04  -2.60433308e-01   5.27890445e+02] Normal Lane left fit
3599.24762212 m: Left Turn Clyde 2611.09042094 m: Right turn Clyde 
[  1.54903853e-04  -2.83718273e-01   5.44873200e+02] Left_Lane.current_fit
[  1.65576057e-04  -2.49351164e-01   5.25955023e+02]
[  1.65576057e-04  -2.49351164e-01   5.25955023e+02] draw lines left_fit
 1.1m from the left lane line
 1.5m from the right lane line
[  1.54903853e-04  -2.83718273e-01   5.44873200e+02] Norma

[  6.84817119e-05  -2.89372410e-01   5.59211236e+02] draw lines left_fit
 1.3m from the left lane line
 1.5m from the right lane line
[ -5.18610839e-05  -2.20827624e-01   5.52073605e+02] Normal Lane left fit
6589.18204031 m: Left Turn Clyde 2497.93319744 m: Right turn Clyde 
[  8.46915130e-05  -2.80704436e-01   5.57162420e+02] Left_Lane.current_fit
IRRATIC BEHAVIOR DETECTED!
[  3.19430364e-05  -2.74482406e-01   5.59948962e+02]
[  3.19430364e-05  -2.74482406e-01   5.59948962e+02] draw lines left_fit
 1.4m from the left lane line
 1.5m from the right lane line
[  1.14399869e-04  -3.10052779e-01   5.68703601e+02] Window left lane fit
4986.96373982 m: Left Turn Clyde 2333.09033215 m: Right turn Clyde 
[  1.14399869e-04  -3.10052779e-01   5.68703601e+02] Left_Lane.current_fit
[  9.34064539e-06  -2.62647865e-01   5.59570191e+02]
[  9.34064539e-06  -2.62647865e-01   5.59570191e+02] draw lines left_fit
 1.4m from the left lane line
 1.5m from the right lane line
[  1.14399869e-04  -3.10052779e

 1.6m from the right lane line
[  1.99090296e-04  -4.13532298e-01   6.28883357e+02] Normal Lane left fit
3321.85378016 m: Left Turn Clyde 2321.21482126 m: Right turn Clyde 
[  1.85699699e-04  -3.98823398e-01   6.24357197e+02] Left_Lane.current_fit
[  2.00577061e-04  -3.55696041e-01   5.87991881e+02]
[  2.00577061e-04  -3.55696041e-01   5.87991881e+02] draw lines left_fit
 1.1m from the left lane line
 1.6m from the right lane line
[  1.85699699e-04  -3.98823398e-01   6.24357197e+02] Normal Lane left fit
3055.34842988 m: Left Turn Clyde 2122.2870023 m: Right turn Clyde 
[  2.09306068e-04  -4.34772655e-01   6.41404860e+02] Left_Lane.current_fit
[  2.04292222e-04  -3.70531328e-01   5.97072049e+02]
[  2.04292222e-04  -3.70531328e-01   5.97072049e+02] draw lines left_fit
 1.1m from the left lane line
 1.6m from the right lane line
[  2.09306068e-04  -4.34772655e-01   6.41404860e+02] Normal Lane left fit
3474.03002821 m: Left Turn Clyde 2310.83760341 m: Right turn Clyde 
[  1.78111742e-04  -

21172.835026 m: Left Turn Clyde 14309.2371562 m: Right turn Clyde 
[  2.40895019e-05  -1.16969557e-01   5.26478144e+02] Left_Lane.current_fit
[  3.99437901e-05  -1.45597913e-01   5.37117649e+02]
[  3.99437901e-05  -1.45597913e-01   5.37117649e+02] draw lines left_fit
 1.0m from the left lane line
 1.7m from the right lane line
[  2.40895019e-05  -1.16969557e-01   5.26478144e+02] Normal Lane left fit
21197.2990796 m: Left Turn Clyde 12639.9409364 m: Right turn Clyde 
[  2.40410671e-05  -1.14431268e-01   5.25893682e+02] Left_Lane.current_fit
IRRATIC BEHAVIOR DETECTED!
[  3.84443944e-05  -1.40533438e-01   5.34831231e+02]
[  3.84443944e-05  -1.40533438e-01   5.34831231e+02] draw lines left_fit
 1.0m from the left lane line
 1.7m from the right lane line
[ -1.20934549e-05  -9.63980159e-02   5.25087564e+02] Window left lane fit
41931.0495839 m: Left Turn Clyde 11252.6375795 m: Right turn Clyde 
[ -1.20934549e-05  -9.63980159e-02   5.25087564e+02] Left_Lane.current_fit
IRRATIC BEHAVIOR DETECT